In [1]:
import numpy as np
import math
from tqdm.notebook import tqdm
from functools import cache
import copy

import heapq



In [2]:
with open("input_day_18.txt") as f:
    text = f.read()


test = False
if test:
    text = r"""
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
"""

text = text.strip()


In [3]:
R = 1000
C = 1000
curr_pos = (R // 2, C // 2)

if test:
    R = 12
    C = 12
    curr_pos = (1,1)

G = [[0 for c in range(C)] for r in range(R)]



In [4]:
G[curr_pos[0]][curr_pos[1]] = 1

for line in text.split("\n"):

    d, n, c = line.split(" ")
    n = int(n)

    if d == "R":
        for nn in range(n):
            curr_pos = (curr_pos[0], curr_pos[1] + 1)
            G[curr_pos[0]][curr_pos[1]] = 1
    if d == "L":
        for nn in range(n):
            curr_pos = (curr_pos[0], curr_pos[1] - 1)
            G[curr_pos[0]][curr_pos[1]] = 1
    if d == "U":
        for nn in range(n):
            curr_pos = (curr_pos[0] - 1, curr_pos[1])
            G[curr_pos[0]][curr_pos[1]] = 1
    if d == "D":
        for nn in range(n):
            curr_pos = (curr_pos[0] + 1, curr_pos[1])
            G[curr_pos[0]][curr_pos[1]] = 1

tot = 0
for row in G:
    tot += sum(row)

print("num border:", tot)

num border: 4688


In [5]:
num_inside = 0

for r in range(R):
    
    inside = False
    on_edge = False
    above_inside = False
    below_inside = False
    
    for c in range(C):


        if on_edge == False and G[r][c] == 1: # entering region

            num_inside += 1
            inside = not inside
            if G[r][c+1] == 1: # will stay in edge, one of the special cases
                on_edge = True
                if G[r-1][c] == 1:
                    above_inside = inside
                    below_inside = not inside
                if G[r+1][c] == 1:
                    below_inside = inside
                    above_inside = not inside

        elif on_edge and G[r][c] == 1: # on border
            num_inside += 1
            if G[r][c+1] == 0: # about to exit edge
                on_edge = False
                if above_inside:
                    if G[r-1][c] == 1:
                        above_inside = False
                        inside = False
                    else:
                        above_inside = True
                        inside = True
                elif below_inside:
                    if G[r+1][c] == 1:
                        below_inside = False
                        inside = False
                    else:
                        below_inside = True
                        inside = True

        elif G[r][c] == 1:
            num_inside += 1

            inside = not inside
            on_edge = False

        elif inside:
            num_inside += 1
            on_edge = False
        else:
            pass
            
print("num filled:", num_inside)


num filled: 92758


In [6]:
keypoints_c = set()
keypoints_r = set()

curr_c = 0
curr_r = 0

for line in text.split("\n"):

    d_, n_, c = line.split(" ")
    n = int(c[2:7], 16)
    d_num = int(c[7])
    d = {0:"R", 1:"D", 2:"L", 3:"U"}[d_num]

    #print(d, n)
    
    if d == "R":
        curr_c += n
    elif d == "L":
        curr_c -= n
    elif d == "U":
        curr_r -= n
    elif d == "D":
        curr_r += n

    keypoints_c.add(curr_c)
    keypoints_c.add(curr_c+1)
    keypoints_r.add(curr_r)
    keypoints_r.add(curr_r+1)

keypoints_c = sorted(list(keypoints_c))
keypoints_r = sorted(list(keypoints_r))

c_widths = [keypoints_c[i+1] - keypoints_c[i] for i in range(len(keypoints_c)-1)]
r_widths = [keypoints_r[i+1] - keypoints_r[i] for i in range(len(keypoints_r)-1)]

#print(keypoints_c)
#print(keypoints_r)

In [7]:
R = 2000
C = 2000
start_pos = (R // 2, C // 2)
#if test:
#    R = 30
#    C = 30
#    start_pos = (4,4)


curr_pos = start_pos
curr_pos_big = (0, 0)
start_index_r = keypoints_r.index(0)
start_index_c = keypoints_c.index(0)

curr_index_r = start_index_r
curr_index_c = start_index_c

G = [[0 for c in range(C)] for r in range(R)]

G[curr_pos[0]][curr_pos[1]] = 1


for line in text.split("\n"):

    d_, n_, c = line.split(" ")
    n = int(c[2:7], 16)
    d_num = int(c[7])
    d = {0:"R", 1:"D", 2:"L", 3:"U"}[d_num]

    if d == "R":
        curr_pos_big = (curr_pos_big[0], curr_pos_big[1] + n)
        while n > 0:
            curr_pos = (curr_pos[0], curr_pos[1] + 1)
            G[curr_pos[0]][curr_pos[1]] = 1
            dist = c_widths[curr_index_c]
            curr_index_c += 1
            n -= dist
    if d == "L":
        curr_pos_big = (curr_pos_big[0], curr_pos_big[1] - n)
        while n > 0:
            curr_pos = (curr_pos[0], curr_pos[1] - 1)
            G[curr_pos[0]][curr_pos[1]] = 1
            dist = c_widths[curr_index_c - 1]
            curr_index_c -= 1
            n -= dist

    if d == "U":
        curr_pos_big = (curr_pos_big[0] - n, curr_pos_big[1])
        while n > 0:
            curr_pos = (curr_pos[0] - 1, curr_pos[1])
            G[curr_pos[0]][curr_pos[1]] = 1
            dist = r_widths[curr_index_r - 1]
            curr_index_r -= 1
            n -= dist
    if d == "D":
        curr_pos_big = (curr_pos_big[0] + n, curr_pos_big[1])
        while n > 0:
            curr_pos = (curr_pos[0] + 1, curr_pos[1])
            G[curr_pos[0]][curr_pos[1]] = 1
            dist = r_widths[curr_index_r]
            curr_index_r += 1
            n -= dist


In [8]:
num_inside = 0
G_inside = [[0 for c in range(C)] for r in range(R)]

for r in range(R):
    
    inside = False
    on_edge = False
    above_inside = False
    below_inside = False
    
    for c in range(C):

        r_index = r - start_pos[0] + start_index_r
        c_index = c - start_pos[1] + start_index_c
        #print(r_index, c_index)

        if on_edge == False and G[r][c] == 1: # entering region

            num_inside += r_widths[r_index] * c_widths[c_index]
            G_inside[r][c] = 1
            inside = not inside
            if G[r][c+1] == 1: # will stay in edge, one of the special cases
                on_edge = True
                if G[r-1][c] == 1:
                    above_inside = inside
                    below_inside = not inside
                if G[r+1][c] == 1:
                    below_inside = inside
                    above_inside = not inside

        elif on_edge and G[r][c] == 1: # on border
            num_inside += r_widths[r_index] * c_widths[c_index]
            G_inside[r][c] = 1
            if G[r][c+1] == 0: # about to exit edge
                on_edge = False
                if above_inside:
                    if G[r-1][c] == 1:
                        above_inside = False
                        inside = False
                    else:
                        above_inside = True
                        inside = True
                elif below_inside:
                    if G[r+1][c] == 1:
                        below_inside = False
                        inside = False
                    else:
                        below_inside = True
                        inside = True

        elif G[r][c] == 1:
            num_inside += r_widths[r_index] * c_widths[c_index]
            G_inside[r][c] = 1

            inside = not inside
            on_edge = False

        elif inside:
            num_inside += r_widths[r_index] * c_widths[c_index]
            G_inside[r][c] = 1
            on_edge = False
        else:
            pass
            
print("num filled:", num_inside)


num filled: 62762509300678


In [9]:
# 66810623890490 too high

In [10]:
min_r = 999999999
max_r = -999999999
min_c = 999999999
max_c = -999999999
for r, row in enumerate(G):
    if sum(row) == 0:
        continue
    for c in range(len(row)):
        if G[r][c] == 1:
            #print("here")
            min_c = min(min_c, c)
            max_c = max(max_c, c)        
            min_r = min(min_r, r)
            max_r = max(max_r, r)

#print(min_r, max_r, min_c, max_c)

#min_r -= 10

together = True 
separate = False

if together:
    for r in range(min_r, max_r+1):
        row = G[r]
        for c in range(min_c, max_c+1):
            if G_inside[r][c] and not G[r][c]:
                print(".", end="")
            elif G_inside[r][c] and G[r][c]:
                print("#", end="")
            elif not G_inside[r][c] and not G[r][c]:
                print(" ", end="")
            else:
                print("!", end="")
        print("")

if separate:

    for r in range(min_r, max_r+1):
        row = G[r]
        print("".join([" " if _ == 0 else "#" for _ in row][min_c:max_c+1]))

    for r in range(min_r, max_r+1):
        row = G_inside[r]
        print("".join([" " if _ == 0 else "#" for _ in row][min_c:max_c+1]))


                                                                                                                                                                                                                                                                                                                                        #####                                                                                                                                                                                                                      
                                                                                                                                                                                                                                                                                                                                        #...#                                                                                                                       

In [11]:
"""for row in G:
    print("".join([" " if _ == 0 else "#" for _ in row]))

print("")

for row in inside_grid:
    print("".join([" " if _ == 0 else "#" for _ in row]))
"""

'for row in G:\n    print("".join([" " if _ == 0 else "#" for _ in row]))\n\nprint("")\n\nfor row in inside_grid:\n    print("".join([" " if _ == 0 else "#" for _ in row]))\n'

In [12]:
"""more_tests = False
if more_tests:
    G = [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
        [0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
    R = len(G)
    C = len(G[0])

    G = [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
        [0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
    R = len(G)
    C = len(G[0])

    G = [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
        [0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
    R = len(G)
    C = len(G[0])

    G = [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
        [0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 1, 0, 1, 0],
        [0, 1, 0, 1, 1, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
    R = len(G)
    C = len(G[0])
    """

'more_tests = False\nif more_tests:\n    G = [\n        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\n        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],\n        [0, 1, 0, 1, 0, 0, 1, 0, 1, 0],\n        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],\n        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],\n        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],\n        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],\n        [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],\n        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\n    ]\n    R = len(G)\n    C = len(G[0])\n\n    G = [\n        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\n        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],\n        [0, 1, 0, 1, 0, 0, 1, 0, 1, 0],\n        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],\n        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],\n        [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],\n        [0, 1, 0, 1, 0, 0, 1, 0, 1, 0],\n        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],\n        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\n    ]\n    R = len(G)\n    C = len(G[0])\n\n    G = [\n        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\n        [0, 1, 1, 1, 0, 0, 1, 1, 1, 0],\n        [0, 1, 0, 1, 1

In [13]:
"""num_enclosed = 0
for i in tqdm(range(R)):

    encountered_nothing_yet = True
    for j in range(C // 2):

        if (i, j) in pos_history:
            encountered_nothing_yet = False
            num_enclosed += 1
            continue

        if encountered_nothing_yet:
            continue
        

        first_pos = pos_history[0]
        prev_angle = np.arctan2(first_pos[1] - j + 0.001, first_pos[0]-i + 0.001)
        cum_angle = 0
        for pos in pos_history:
            curr_angle = np.arctan2(pos[1] - j + 0.001, pos[0]-i + 0.001)
            angle_diff = curr_angle - prev_angle
            if angle_diff > np.pi / 2: # switched from lower left to upper left quadrant
                angle_diff -= 2 * np.pi
            elif angle_diff < -np.pi / 2: # switched from lower left to upper left quadrant
                angle_diff += 2 * np.pi
            cum_angle += angle_diff
            prev_angle = curr_angle
        #print(cum_angle)

        if abs(cum_angle) > 0.1:
            num_enclosed += 1

    encountered_nothing_yet = True
    for j in range(C-1, C // 2 - 1, -1):

        if (i, j) in pos_history:
            encountered_nothing_yet = False
            num_enclosed += 1
            continue

        if encountered_nothing_yet:
            continue
        

        first_pos = pos_history[0]
        prev_angle = np.arctan2(first_pos[1] - j + 0.001, first_pos[0]-i + 0.001)
        cum_angle = 0
        for pos in pos_history:
            curr_angle = np.arctan2(pos[1] - j + 0.001, pos[0]-i + 0.001)
            angle_diff = curr_angle - prev_angle
            if angle_diff > np.pi / 2: # switched from lower left to upper left quadrant
                angle_diff -= 2 * np.pi
            elif angle_diff < -np.pi / 2: # switched from lower left to upper left quadrant
                angle_diff += 2 * np.pi
            cum_angle += angle_diff
            prev_angle = curr_angle
        #print(cum_angle)

        if abs(cum_angle) > 0.1:
            num_enclosed += 1


num_enclosed
"""


'num_enclosed = 0\nfor i in tqdm(range(R)):\n\n    encountered_nothing_yet = True\n    for j in range(C // 2):\n\n        if (i, j) in pos_history:\n            encountered_nothing_yet = False\n            num_enclosed += 1\n            continue\n\n        if encountered_nothing_yet:\n            continue\n        \n\n        first_pos = pos_history[0]\n        prev_angle = np.arctan2(first_pos[1] - j + 0.001, first_pos[0]-i + 0.001)\n        cum_angle = 0\n        for pos in pos_history:\n            curr_angle = np.arctan2(pos[1] - j + 0.001, pos[0]-i + 0.001)\n            angle_diff = curr_angle - prev_angle\n            if angle_diff > np.pi / 2: # switched from lower left to upper left quadrant\n                angle_diff -= 2 * np.pi\n            elif angle_diff < -np.pi / 2: # switched from lower left to upper left quadrant\n                angle_diff += 2 * np.pi\n            cum_angle += angle_diff\n            prev_angle = curr_angle\n        #print(cum_angle)\n\n        if a